<a href="https://colab.research.google.com/github/anirudhh004/PriorityPredictionAgent/blob/main/model/PriorityPredictionNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
df = pd.read_csv('/content/sample_data/priority_agent_dataset.csv')
df.head()

,Deadline,Task_type,Estimated_effort,Impact,Urgency_flag,Priority_score
0,52,feature,10,high,0,0.41
1,15,personal,1,high,0,0.61
2,72,personal,6,medium,1,0.24
3,61,bug,6,high,0,0.55
4,21,feature,4,low,0,0.44


In [ ]:
df1 = pd.get_dummies(df, columns=['Task_type', 'Impact'], dtype=int)
df1.head()

,Deadline,Estimated_effort,Urgency_flag,Priority_score,Task_type_bug,Task_type_feature,Task_type_personal,Impact_high,Impact_low,Impact_medium
0,52,10,0,0.41,0,1,0,1,0,0
1,15,1,0,0.61,0,0,1,1,0,0
2,72,6,1,0.24,0,0,1,0,0,1
3,61,6,0,0.55,1,0,0,1,0,0
4,21,4,0,0.44,0,1,0,0,1,0


In [ ]:
X = df1.drop(columns=['Priority_score'])
y = df1['Priority_score']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
type(X_train)

pandas.core.frame.DataFrame

In [ ]:
num_cols = ['Deadline', 'Estimated_effort']

In [ ]:
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [ ]:
X_train


,Deadline,Estimated_effort,Urgency_flag,Task_type_bug,Task_type_feature,Task_type_personal,Impact_high,Impact_low,Impact_medium
249,-0.305305,1.506744,0,0,1,0,1,0,0
433,-0.591864,-0.186227,1,1,0,0,0,1,0
19,0.602132,1.506744,1,0,0,1,0,0,1
322,-0.544104,-1.202009,1,0,0,1,0,0,1
332,1.605089,-0.186227,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...
106,-1.069463,-1.540604,0,0,1,0,0,1,0
270,-0.305305,0.152367,1,0,0,1,1,0,0
348,-0.305305,1.506744,0,0,1,0,1,0,0
435,0.602132,0.490962,0,0,1,0,1,0,0


In [ ]:
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()

In [ ]:
print(X_train_np.shape, X_test_np.shape)
print(y_train_np.shape, y_test_np.shape)


(400, 9) (100, 9)
(400,) (100,)


In [ ]:
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_np, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_np, dtype=torch.float32)


In [ ]:
y_train_tensor = y_train_tensor.view(-1,1)
y_test_tensor = y_test_tensor.view(-1,1)


In [ ]:
print(X_train_tensor.shape, X_test_tensor.shape)
print(y_train_tensor.shape, y_test_tensor.shape)


torch.Size([400, 9]) torch.Size([100, 9])
torch.Size([400, 1]) torch.Size([100, 1])


In [ ]:
print(X_train_np.dtype)  # prints the data type of array elements
print(y_train_np.dtype)

float64
float64


In [ ]:
class PriorityNN(nn.Module):
    def __init__(self, input_size):
        super(PriorityNN, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_size, 32),  # WX + b
            nn.ReLU(),                  # non-linearity

            nn.Linear(32, 16),
            nn.ReLU(),

            nn.Linear(16, 1),
        )

    def forward(self, x):
        return self.model(x)

Important Parameters


In [ ]:
learning_rate = 0.01
epochs = 25

In [ ]:
loss_function = nn.MSELoss()

Training Pipeline

In [ ]:
# create model
model = PriorityNN(X_train_tensor.shape[1])

#define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensor)

  # loss calculate
  loss = loss_function(y_pred, y_train_tensor)

  # zero gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.4374839663505554
Epoch: 2, Loss: 0.3084254860877991
Epoch: 3, Loss: 0.20985765755176544
Epoch: 4, Loss: 0.1340157687664032
Epoch: 5, Loss: 0.07820392400026321
Epoch: 6, Loss: 0.0411003977060318
Epoch: 7, Loss: 0.024193501099944115
Epoch: 8, Loss: 0.027962539345026016
Epoch: 9, Loss: 0.04302344471216202
Epoch: 10, Loss: 0.05243007838726044
Epoch: 11, Loss: 0.049434490501880646
Epoch: 12, Loss: 0.038289304822683334
Epoch: 13, Loss: 0.02563449926674366
Epoch: 14, Loss: 0.015931909903883934
Epoch: 15, Loss: 0.01058899611234665
Epoch: 16, Loss: 0.00903535820543766
Epoch: 17, Loss: 0.00980351958423853
Epoch: 18, Loss: 0.011419816873967648
Epoch: 19, Loss: 0.0128225889056921
Epoch: 20, Loss: 0.013408913277089596
Epoch: 21, Loss: 0.012981558218598366
Epoch: 22, Loss: 0.011619389988481998
Epoch: 23, Loss: 0.009580609388649464
Epoch: 24, Loss: 0.007258005905896425
Epoch: 25, Loss: 0.0051100002601742744


In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)

    mse = torch.mean((y_pred - y_test_tensor) ** 2)
    rmse = torch.sqrt(mse)
    mae = torch.mean(torch.abs(y_pred - y_test_tensor))

print(f"MSE  : {mse.item():.6f}")
print(f"RMSE : {rmse.item():.6f}")
print(f"MAE  : {mae.item():.6f}")



MSE  : 0.003473
RMSE : 0.058935
MAE  : 0.046921


In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_train_tensor)

    mse = torch.mean((y_pred - y_train_tensor) ** 2)
    rmse = torch.sqrt(mse)
    mae = torch.mean(torch.abs(y_pred - y_train_tensor))

print(f"MSE  : {mse.item():.6f}")
print(f"RMSE : {rmse.item():.6f}")
print(f"MAE  : {mae.item():.6f}")


MSE  : 0.003565
RMSE : 0.059705
MAE  : 0.048333


In [ ]:
for i in range(5):
    print(f"Predicted: {y_pred[i].item():.3f} | Actual: {y_test_tensor[i].item():.3f}")

Predicted: 0.494 | Actual: 0.610
Predicted: 0.592 | Actual: 0.550
Predicted: 0.324 | Actual: 0.450
Predicted: 0.531 | Actual: 0.330
Predicted: 0.257 | Actual: 0.430
